In [ ]:
%matplotlib inline

import logging

import emcee
import h5py
import numpy as np
import matplotlib.pyplot as plt

from os import path
from glob import glob
import sys
sys.path.insert(0, path.abspath('./'))

from src import workdir, parse_model_parameter_file
from src.emulator_BAND import EmulatorBAND

In [ ]:
model_par = "../data/modelDesign_3DMCGlauber.txt"
training_set = "../data/AuAu7.7_dNdy.pkl"
emu = EmulatorBAND(training_set, model_par, npc=3, logFlag=False)#npc is dummy here

In [ ]:
output = emu.testEmulatorErrors(number_test_points=400, number_iterations=1)

emu_pred = output[0]
emu_pred_err = output[1]
vali_data = output[2]
vali_data_err = output[3]

In [ ]:
nValidationPoints, nObs = vali_data.shape

for obsIdx in range(nObs):
    fig = plt.figure()
    ax = plt.axes([0.12, 0.12, 0.83, 0.83])
    plt.errorbar(vali_data[:, obsIdx], emu_pred[:, obsIdx],
                 yerr=emu_pred_err[:, obsIdx],
                 xerr=vali_data_err[:, obsIdx],
                 marker="o", linestyle="")
    plt.plot([-200, 200], [-200, 200], '--k')
    plt.xlim([vali_data[:, obsIdx].min() - 1,
              vali_data[:, obsIdx].max() + 1])
    plt.ylim([emu_pred[:, obsIdx].min() - 1,
              emu_pred[:, obsIdx].max() + 1])
    ax.set_aspect('equal')
    plt.text(vali_data[:, obsIdx].min(),
             emu_pred[:, obsIdx].max(),
             "obs {}".format(obsIdx), fontsize=20)
    plt.xlabel("truth")
    plt.ylabel("emulator results")

In [ ]:
nValidationPoints, nObs = vali_data.shape

for obsIdx in range(nObs):
    fig = plt.figure()
    ax = plt.axes([0.12, 0.12, 0.83, 0.83])
    plt.plot(vali_data[:, obsIdx],
             (emu_pred[:, obsIdx] - vali_data[:, obsIdx])
             /np.sqrt(vali_data_err[:, obsIdx]**2.
                      + emu_pred_err[:, obsIdx]**2.),
             marker="o", linestyle="")
    plt.fill_between([-20, 20], [2, 2], [-2, -2], color='g', alpha=0.2)
    plt.xlim([vali_data[:, obsIdx].min() - 1,
              vali_data[:, obsIdx].max() + 1])
    plt.ylim([-4, 4])
    plt.text(vali_data[:, obsIdx].min(), 3,
             "obs {}".format(obsIdx), fontsize=20)
    plt.xlabel("truth")
    plt.ylabel("relative diff. [$\sigma$]")

In [ ]:
nValidationPoints, nObs = vali_data.shape

for obsIdx in range(nObs):
    fig = plt.figure()
    ax = plt.axes([0.12, 0.12, 0.83, 0.83])
    plt.hist((emu_pred[:, obsIdx]-vali_data[:, obsIdx])/emu_pred_err[:, obsIdx], density=True)
    #plt.text(0, 3,"obs {}".format(obsIdx), fontsize=20)
    plt.xlabel("A = (GP-truth)/GPerr")
    plt.ylabel("p(A)")